<p> <center> <a href="../start_here.ipynb">Home Page</a> </center> </p>

<div>
    <span style="float: left; width: 33%; text-align: left;"><a href="01_inference_endpoint.ipynb">Previous Notebook</a></span>
    <span style="float: left; width: 34%; text-align: center;">
        <a href="01_inference_endpoint.ipynb">1</a>
        <a >2</a>
        <a href="03_low_level_mcp.ipynb">3</a>
        <a href="04_langraph.ipynb">4</a>
        <a href="05_challenge.ipynb">5</a>
    </span>
    <span style="float: left; width: 33%; text-align: right;"><a href="03_low_level_mcp.ipynb">Next Notebook</a></span>
</div>

## Learning Objectives
By the end of this notebook, participants will be able to:
* Understand the Model Context Protocol (MCP) as an open standard for connecting AI assistants to external tools and data sources
* Build MCP servers using the FastMCP high-level SDK with custom tool definitions
* Deploy and connect to MCP servers using two approaches:
  - **Claude Desktop Integration**: Configure MCP servers with Stdio transport
  - **Programmatic Access**: Run HTTP servers and connect via Python MCP client
* Create MCP clients to discover and invoke tools programmatically
* Manage server processes (start, monitor, and terminate)
* Recognize security considerations when integrating third-party MCP servers

## Introduction to Model Context Protocol (MCP)

**Key Components:**
- **MCP Server**: Exposes tools (functions) that can be called by clients
- **MCP Client**: Discovers and invokes tools from MCP servers
- **Transport**: Communication layer (Stdio or HTTP)

## Create the MCP Server

First, let's create a simple MCP server that exposes math tools (add, subtract).

In [ ]:
%%writefile mcp_server.py
"""
MCP Server using FastMCP (High-Level SDK)
This server exposes simple math tools that can be invoked by MCP clients.
"""

from mcp.server.fastmcp import FastMCP

# Create an MCP server instance with a descriptive name
mcp = FastMCP("simple-math")

# Define the 'add' tool using the @mcp.tool() decorator
@mcp.tool()
def add(a: int, b: int) -> int:
    """Add two numbers together.
    
    Args:
        a: First integer
        b: Second integer
    
    Returns:
        Sum of a and b
    """
    return a + b

# Define the 'subtract' tool
@mcp.tool()
def subtract(a: int, b: int) -> int:
    """Subtract second number from first.
    
    Args:
        a: First integer
        b: Second integer
    
    Returns:
        Difference (a - b)
    """
    return a - b

if __name__ == "__main__":
    # Run the MCP server with SSE transport over HTTP
    mcp.run(transport="stdio")

## Running MCP Server on Your Local Machine

Accessing Connectors in Claude Desktop
- Navigate to https://claude.com/product/overview
- Click the + (plus) button
- Select Connectors from the menu
- Choose Manage Connectors

<div style="text-align: center;">
  <img src="images/claude-connectors.png" style="width: 600px; height: auto;">
</div>

Editing Configuration
- Navigate to Developer settings
- Select Edit Config
- Modify the `claude_desktop_config.json` file and add your Claude Desktop

<div style="text-align: center;">
  <img src="images/claude-developer.png" style="width: 600px; height: auto;">
</div>

After Configuration:
- Save the `claude_desktop_config.json` file
- Restart Claude Desktop (quit completely and relaunch), make sure there are no errors
- Verify connection in Settings → Manage Connectors
- Validate the output

<div style="text-align: center;">
  <img src="images/claude-tools.png" style="width: 600px; height: auto;">
</div>

<div style="text-align: center;">
  <img src="images/claude.png" style="width: 600px; height: auto;">
</div>

Please note that the free plan has token usage limitations. For the assignment, you can test your MCP servers using a similar configuration to what we've demonstrated above.

To thoroughly validate your workflow and MCP servers, consider generating synthetic test cases. This approach allows you to simulate various scenarios and ensure your implementation works correctly across different use cases.

## Programmatic MCP Server Access

In this section, we'll launch the MCP server as a **background subprocess**, allowing us to connect and invoke tools programmatically using a Python client—no Claude Desktop required.

We'll launch the server as a background process using `subprocess.Popen()`. This allows us to:
- Keep the server running while executing other cells
- Track the process ID (PID) for cleanup later

In [ ]:
import subprocess
import time

# Start the HTTP server in the background
server_process = subprocess.Popen(
    ["python", "mcp_server.py"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for server to start
time.sleep(3)

# Store and display the process ID
SERVER_PID = server_process.pid
print(f"Process ID: {SERVER_PID}")

### MCP Client Class with Dual Transport Support

Let's understand the `MCPClient` class that connects to MCP servers. Unlike server implementations, here we need to handle connection, tool discovery, and invocation:

1. **Initialize** `__init__()` – Set up session placeholder and `AsyncExitStack` for resource management
2. **Define** `connect_to_server()` – Launch and connect to local server via Stdio transport
3. **Define** `connect_to_server_http()` – Connect to remote server via HTTP transport
4. **Define** `list_tools()` – Discover available tools from the connected server
5. **Define** `call_tool()` – Execute a specific tool with arguments
6. **Define** `cleanup()` – Release all resources by closing the `AsyncExitStack`

In [ ]:
%%writefile mcp_client.py
"""
MCP Client that connects to and invokes tools from MCP servers.

Supports:
- Stdio transport (for local servers)
- HTTP transport (for remote/HTTP servers)

Usage:
    python mcp_client.py                           # Default: mcp_server.py (stdio)
    python mcp_client.py mcp_server.py             # High-level server (stdio)
    python mcp_client.py mcp_server_low_level.py   # Low-level server (stdio)
    python mcp_client.py --http                    # HTTP mode (localhost:8000)
    python mcp_client.py --http http://custom:8080/mcp  # Custom HTTP URL
"""

import asyncio
import argparse
from typing import Optional
from contextlib import AsyncExitStack
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from mcp.client.streamable_http import streamablehttp_client


class MCPClient:
    """A client for connecting to MCP servers via Stdio or HTTP transport."""
    
    # ===========================================
    # Step 1: Initialize __init__()
    # Set up session placeholder and AsyncExitStack for resource management
    # ===========================================
    def __init__(self):
        self.session: Optional[ClientSession] = None
        self.exit_stack = AsyncExitStack()
    
    # ===========================================
    # Step 2: Define connect_to_server()
    # Launch and connect to local server via Stdio transport
    # ===========================================
    async def connect_to_server(self, server_script_path: str):
        print(f"→ Connecting to server: {server_script_path}")
        server_params = StdioServerParameters(
            command="python",
            args=[server_script_path]
        )
        stdio_transport = await self.exit_stack.enter_async_context(
            stdio_client(server_params)
        )
        self.stdio, self.write = stdio_transport
        self.session = await self.exit_stack.enter_async_context(
            ClientSession(self.stdio, self.write)
        )
        await self.session.initialize()
    
    # ===========================================
    # Step 3: Define connect_to_server_http()
    # Connect to remote server via HTTP transport
    # ===========================================
    async def connect_to_server_http(self, url: str):
        print(f"→ Connecting to HTTP server: {url}")
        streamablehttp_transport = await self.exit_stack.enter_async_context(
            streamablehttp_client(url)
        )
        self.read, self.write, _ = streamablehttp_transport
        self.session = await self.exit_stack.enter_async_context(
            ClientSession(self.read, self.write)
        )
        await self.session.initialize()
    
    # ===========================================
    # Step 4: Define list_tools()
    # Discover available tools from the connected server
    # ===========================================
    async def list_tools(self):
        res_tools = await self.session.list_tools()
        return res_tools.tools
    
    # ===========================================
    # Step 5: Define call_tool()
    # Execute a specific tool with arguments
    # ===========================================
    async def call_tool(self, tool_name: str, arguments: dict):
        result = await self.session.call_tool(tool_name, arguments)
        return result.content[0].text
    
    # ===========================================
    # Step 6: Define cleanup()
    # Release all resources by closing the AsyncExitStack
    # ===========================================
    async def cleanup(self):
        await self.exit_stack.aclose()


async def test_tools(client: MCPClient):
    """Test the available tools on the connected server."""
    tools = await client.list_tools()
    print(f"\n✓ Connected to server with tools: {[tool.name for tool in tools]}")
    
    print("\n→ Calling add(2, 3)")
    result = await client.call_tool('add', {'a': 2, 'b': 3})
    print(f"  Result: {result}")
    
    print("\n→ Calling subtract(7, 4)")
    result = await client.call_tool('subtract', {'a': 7, 'b': 4})
    print(f"  Result: {result}")


async def main_stdio(server_path: str):
    """Run client with Stdio transport."""
    client = MCPClient()
    try:
        await client.connect_to_server(server_path)
        await test_tools(client)
    finally:
        await client.cleanup()


async def main_http(url: str):
    """Run client with HTTP transport."""
    client = MCPClient()
    try:
        await client.connect_to_server_http(url)
        await test_tools(client)
    finally:
        await client.cleanup()


if __name__ == '__main__':
    parser = argparse.ArgumentParser(
        description="MCP Client - Connect to MCP servers and invoke tools"
    )
    parser.add_argument(
        'server',
        nargs='?',
        default='mcp_server.py',
        help='Path to server script (default: mcp_server.py)'
    )
    parser.add_argument(
        '--http',
        nargs='?',
        const='http://localhost:8000/mcp',
        help='Use HTTP transport (default URL: http://localhost:8000/mcp)'
    )
    
    args = parser.parse_args()
    
    if args.http:
        url = args.http
        asyncio.run(main_http(url))
    else:
        asyncio.run(main_stdio(args.server))

Let's run the client to connect and invoke the tools.

In [ ]:
!python mcp_client.py

Always clean up background processes when you're done. We'll terminate the server using the PID we saved earlier.

In [ ]:
!kill -9 {SERVER_PID}
print(f"✓ Server (PID: {SERVER_PID}) killed")

### Further Exploration

Try extending the MCP server with additional operations:

- Add a `multiply(a, b)` tool
- Add a `divide(a, b)` tool (hint: handle division by zero!)
- Switch to HTTP transport using `mcp.run(transport='streamable-http')` and use `mcp_client.py` to connect. 

### Links and Resources

- [Model Context Protocol](https://modelcontextprotocol.io/docs/getting-started/intro)
- [Claude](https://claude.com/product/overview)
- [FastMCP](https://github.com/jlowin/fastmcp)

---

## Licensing

Copyright © 2025 OpenACC-Standard.org. This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.

<p> <center> <a href="../start_here.ipynb">Home Page</a> </center> </p>

<div>
    <span style="float: left; width: 33%; text-align: left;"><a href="rag_nim_endpoints.ipynb">Previous Notebook</a></span>
    <span style="float: left; width: 34%; text-align: center;">
        <a href="01_inference_endpoint.ipynb">1</a>
        <a >2</a>
        <a href="03_low_level_mcp.ipynb">3</a>
        <a href="04_langraph_agent.ipynb">4</a>
        <a href="05_challenge.ipynb">5</a>
    </span>
    <span style="float: left; width: 33%; text-align: right;"><a href="03_low_level_mcp.ipynb">Next Notebook</a></span>
</div>